In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import tensorflow as tf
tf.enable_eager_execution()
tfe = tf.contrib.eager

In [2]:
import warnings
warnings.filterwarnings("ignore")
from collections import defaultdict
import time
from multiprocessing import Pool
import numpy as np
import pandas as pd
from math import sqrt
from pdb import set_trace
import tensorflow as tf

In [3]:
np.random.seed(42)
train_set = open("ml-1m/ratings.dat").readlines()
train_set = np.random.permutation(train_set)[200000:400000]
train_threshold = int(0.8 * len(train_set))
# train_indices = []
# test_indices = []
train_user_indices = []
train_item_indices = []
test_user_indices = []
test_item_indices = []
train_ratings = []
test_ratings = []

data = defaultdict(dict)
user2id = {}
item2id = {}
index_user = 0
index_item = 0

for i, line in enumerate(train_set):
    user = line.split("::")[0]
    item = line.split("::")[1]
    rating = line.split("::")[2]
    
    try:
        user_id = user2id[user]
    except KeyError:
        user_id = index_user
        user2id[user] = index_user
        index_user += 1
    try:
        item_id = item2id[item]
    except KeyError:
        item_id = index_item
        item2id[item] = index_item
        index_item += 1
        
    if i < train_threshold:
    #    train_indices.append((user_id, item_id))
        train_user_indices.append(user_id)
        train_item_indices.append(item_id)
        train_ratings.append(int(rating))
        data[user_id].update(dict(zip([item_id], [int(rating)])))
    else:
    #    test_indices.append((user_id, item_id))
        test_user_indices.append(user_id)
        test_item_indices.append(item_id)
        test_ratings.append(int(rating))

In [4]:
len(train_ratings), len(test_ratings)

(160000, 40000)

In [5]:
print("before: ", len(train_ratings), len(test_ratings))

for u, i, r in zip(test_user_indices, test_item_indices, test_ratings):
    if u not in data:
        test_user_indices.remove(u)
        test_item_indices.remove(i)
        test_ratings.remove(r)

for u, i, r in zip(test_user_indices, test_item_indices, test_ratings):
    if u not in data:
        test_user_indices.remove(u)
        test_item_indices.remove(i)
        test_ratings.remove(r)

for u, i, r in zip(test_user_indices, test_item_indices, test_ratings):
    if u not in data:
        test_user_indices.remove(u)
        test_item_indices.remove(i)
        test_ratings.remove(r)

for u in test_user_indices:
    if u not in data:
        print("left", u)

print("after: ", len(train_ratings), len(test_ratings))

before:  160000 40000
after:  160000 39967


In [6]:
def ratings(dataset):
    for user, r in data.items():
        for item, rating in r.items():
            yield user, item, rating

### calculate bbu & bbi 

In [7]:
def compute_rmse(mode="train"):
    if mode == "train":
        user_indices = train_user_indices
        item_indices = train_item_indices
        ratings = train_ratings
    elif mode == "test":
        user_indices = test_user_indices
        item_indices = test_item_indices
        ratings = test_ratings
        
    global_mean = np.mean(train_ratings)
    pred = global_mean + bbu[user_indices] + bbi[item_indices] 
    score = np.sqrt(np.mean(np.power(pred - ratings, 2)))
    return score

In [8]:
np.random.seed(42)
n_users = len(user2id)
n_items = len(item2id)
global_mean = np.mean(train_ratings)
n_epochs = 200
lr = 0.002
reg = 0.1
bbu = np.zeros((n_users,))
bbi = np.zeros((n_items,))
for epoch in range(n_epochs):
    for u, i, r in ratings(data):
        before = bbu.copy()
        err = r - (global_mean + bbu[u] + bbi[i])
        bbu[u] += lr * (err - reg * bbu[u])
        bbi[i] += lr * (err - reg * bbi[i])
    if epoch % 5 == 0:
        print("Epoch: ", epoch + 1, "training RMSE: ", compute_rmse("train"), "test RMSE: ", compute_rmse("test"))
        if np.allclose(before, bbu, atol=1e-3):
            print("\n Converged !")
            break

Epoch:  1 training RMSE:  1.1038086908163407 test RMSE:  1.0985982825866456

 Converged !


### neighborhood model

In [8]:
start = time.time()
np.random.seed(42)
tf.reset_default_graph()
n_factors = 100
n_users = len(user2id)
n_items = len(item2id)
global_mean = np.mean(train_ratings)
base_user = bbu
base_item = bbi

n_epochs = 500
lr = 0.01
reg = 1e-4
display_step = 1

bu = tf.Variable(tf.zeros([n_users]))
bi = tf.Variable(tf.zeros([n_items]))
w = tf.Variable(tf.random_normal([n_items, n_items], 0.0, 0.01))
c = tf.Variable(tf.random_normal([n_items, n_items], 0.0, 0.01))

user_indices = tf.placeholder(tf.int32, shape=[None])
item_indices = tf.placeholder(tf.int32, shape=[None])
ratings = tf.placeholder(tf.int32, shape=[None])


expanded_items = np.zeros((n_users, n_items), dtype=np.int32)
for u in data:
    u_items = list(data[u].keys())
    expanded_items[u] = np.array(u_items + [-1] * (n_items - len(u_items)))
    
expanded_ratings = np.zeros((n_users, n_items), dtype=np.int32)
for u in data:
    u_ratings = list(data[u].values())
    expanded_ratings[u] = np.array(u_ratings + [-1] * (n_items - len(u_ratings)))

def predict_func(elem):
    u = elem[0]
    i = elem[1]
#    r = elem[2]
    mask = tf.constant(-1, dtype=tf.int32)
    u_expanded_items = tf.nn.embedding_lookup(expanded_items, u)
    u_items = tf.boolean_mask(u_expanded_items, tf.not_equal(u_expanded_items, mask))
    
    u_expanded_ratings = tf.nn.embedding_lookup(expanded_ratings, u)
    u_ratings = tf.boolean_mask(u_expanded_ratings, tf.not_equal(u_expanded_ratings, mask))
    
    base_neighbor = global_mean + tf.gather(base_user, u) + tf.gather(base_item, u_items)
    w_ij = tf.gather(tf.gather(w, i), u_items)
    c_ij = tf.gather(tf.gather(c, i), u_items)
    
#    ru = tf.reduce_sum((tf.cast(u_ratings, tf.float32) - tf.cast(base_neighbor, tf.float32)) * 
#                       w_ij) / tf.sqrt(tf.cast(tf.size(u_items), tf.float32))
    u_ratings = tf.cast(u_ratings, tf.float32)
    base_neighbor = tf.cast(base_neighbor, tf.float32)
    ru = tf.reduce_sum(tf.multiply(tf.subtract(u_ratings, base_neighbor), w_ij))
    ru = tf.divide(ru, tf.sqrt(tf.cast(tf.size(u_items), tf.float32)))
    nu = tf.reduce_sum(c_ij) / tf.sqrt(tf.cast(tf.size(u_items), tf.float32))
    pred = global_mean + tf.gather(bu, u) + tf.gather(bi, i) + ru + nu
    return pred
    
# pred = tf.map_fn(predict_func, (np.array(train_user_indices), np.array(train_item_indices)), dtype=tf.float32)
pred = tf.map_fn(predict_func, (user_indices, item_indices), dtype=tf.float32)
pred = tf.convert_to_tensor(pred)
loss = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(tf.cast(ratings, tf.float32), pred))))
    

reg_bu = tf.contrib.layers.l2_regularizer(reg)(bu)
reg_bi = tf.contrib.layers.l2_regularizer(reg)(bi)
reg_w = tf.contrib.layers.l2_regularizer(reg)(w)
reg_c = tf.contrib.layers.l2_regularizer(reg)(c)
total_loss = tf.add_n([loss, reg_bu, reg_bi, reg_w, reg_c])
optimizer = tf.train.AdamOptimizer(lr)
# optimizer = tf.train.GradientDescentOptimizer(lr)
training_op = optimizer.minimize(total_loss)
init = tf.global_variables_initializer()
print("before time", time.time() - start)


with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        t0 = time.time()
        sess.run(training_op, feed_dict={user_indices: train_user_indices, 
                                         item_indices: train_item_indices, 
                                         ratings: train_ratings})
        print("Epoch: ", epoch+1, "training time: ", time.time() - t0)
        t1 = time.time()
        if epoch % display_step == 0:
            train_loss = loss.eval(feed_dict={user_indices: train_user_indices, 
                                              item_indices: train_item_indices, 
                                              ratings: train_ratings})
            test_loss = loss.eval(feed_dict={user_indices: test_user_indices, 
                                             item_indices: test_item_indices, 
                                             ratings: test_ratings})
            print("Epoch: ", epoch + 1, "\ttrain loss: ", train_loss, "\ttest loss: ", test_loss)
            print("evaluate time: ", time.time() - t1)
            print()

before time 9.674000024795532
Epoch:  1 training time:  204.36899995803833
Epoch:  1 	train loss:  1.0718961 	test loss:  1.0989753
evaluate time:  2.684000015258789



KeyboardInterrupt: 

### Eager Execution - neighborhood model

In [6]:
# import pickle
# bbu, bbi = pickle.load(open("bbu_bbi.pkl", 'rb'))

In [19]:
start = time.time()
np.random.seed(42)
tf.reset_default_graph()
n_factors = 100
n_users = len(user2id)
n_items = len(item2id)
global_mean = np.mean(train_ratings)
base_user = bbu
base_item = bbi

n_epochs = 500
lr = 0.01
reg = 1e-4
display_step = 1

bu = tf.Variable(tf.zeros([n_users]))
bi = tf.Variable(tf.zeros([n_items]))
w = tf.Variable(tf.random_normal([n_items, n_items], 0.0, 0.01))
c = tf.Variable(tf.random_normal([n_items, n_items], 0.0, 0.01))


def compute_rmse_neighbor(mode="train"):
    if mode == "train":
        user_indices = train_user_indices
        item_indices = train_item_indices
        ratings = train_ratings
    elif mode == "test":
        user_indices = test_user_indices
        item_indices = test_item_indices
        ratings = test_ratings
        
    score = 0
    pred_whole = []
    for u, i in zip(user_indices, item_indices):
        u_items = list(data[u].keys())
        u_ratings = np.array(list(data[u].values()))
        base_neighbor = global_mean + bbu[u] + bbi[u_items]
#        ru = np.sum((u_ratings - base_neighbor) * w[i][u_items]) / np.sqrt(len(u_items))
#        nu = np.sum(c[i][u_items]) / np.sqrt(len(u_items))
        if len(data[u]) == 0:
            ru = 0
            nu = 0
        else:
            ru = tf.reduce_sum((u_ratings - base_neighbor) * tf.gather(tf.gather(w, i), u_items)) / \
                tf.sqrt(tf.cast(tf.size(u_items), tf.float32))
            nu = tf.reduce_sum(tf.gather(tf.gather(c, i), u_items)) / tf.sqrt(tf.cast(tf.size(u_items), tf.float32))
        pred = global_mean + bu[u] + bi[i] + ru + nu
        pred_whole.append(pred)
    pred_whole = tf.convert_to_tensor(pred_whole)
    return tf.sqrt(tf.reduce_mean(tf.pow(pred_whole - np.array(ratings), 2)))

In [21]:
optimizer = tf.train.GradientDescentOptimizer(lr)
# optimizer = tf.train.AdamOptimizer(lr)
for epoch in range(n_epochs):
    t0 = time.time()
    with tf.GradientTape() as tape:
        variables = [bu, bi, w, c]
        loss = compute_rmse_neighbor()
        grads = tape.gradient(loss, variables)
        optimizer.apply_gradients(zip(grads, variables))
        
    print("Epoch: ", epoch + 1, "\ttime: ", time.time() - t0)
    
    if epoch % 1 == 0:
        t1 = time.time()
        print("Epoch: ", epoch + 1, "\ttraining RMSE: ", compute_rmse_neighbor("train").numpy(), 
              "\ttest RMSE: ", compute_rmse_neighbor("test").numpy())
        print("evaluate time: ", time.time() - t1)

Epoch:  1 	time:  20.796000003814697
Epoch:  1 	training RMSE:  0.91137004 	test RMSE:  1.1005447
evaluate time:  6.771999835968018
Epoch:  2 	time:  19.57800006866455
Epoch:  2 	training RMSE:  0.90764153 	test RMSE:  1.1006138
evaluate time:  6.5920000076293945
Epoch:  3 	time:  23.756999969482422
Epoch:  3 	training RMSE:  0.9039209 	test RMSE:  1.1006826
evaluate time:  6.863000154495239


KeyboardInterrupt: 

### Eager Execution - neighborhood model K

In [7]:
import pickle
bbu, bbi = pickle.load(open("bbu_bbi.pkl", 'rb'))

In [7]:
def cosine_similarity(dicts, x1, x2):  # only common ratings
    prod = denom1 = denom2 = 0
    for item in dicts[x1]:
        if item in dicts[x2]:
            prod += dicts[x1][item] * dicts[x2][item]
            denom1 += dicts[x1][item] ** 2
            denom2 += dicts[x2][item] ** 2
    try:
        return prod / sqrt(denom1 * denom2)
    except ZeroDivisionError:
        return 0
    
def map_func(i):
    return [cosine_similarity(data, i, other) for other in item_idsss]


def get_sim(n, item_ids, parallel=False, symmetric=True, n_jobs=4):
    if not parallel and not symmetric:
        print("not symmetric")
        sim = np.array([[cosine_similarity(data, i, other) for other in item_ids] for i in item_ids])
    if not parallel and symmetric:
        print("symetric")
        sim = np.zeros((n, n))
        for i in item_ids:   ################# item_ids
            sim[i, i] = 1.0
            for j in item_ids[i + 1:]:
                sim[i, j] = cosine_similarity(data, i, j)
                sim[j, i] = sim[i, j]
    if parallel:
        print("multiprocessing")
        with Pool(processes=n_jobs) as p:
            sim = p.map(map_func, item_ids)
            sim = np.array(sim)
    return sim

item_idsss = list(item2id.values())   # 排除 test data
n_items = len(item2id)
%time sim_matrix = get_sim(n_items, item_idsss)

symetric
CPU times: user 40.1 s, sys: 228 ms, total: 40.3 s
Wall time: 40.3 s


In [8]:
sim_whole = {}
for i in range(n_items):
    sim_whole[i] = np.argsort(sim_matrix[i])[::-1]
    
k = 20
intersect_user_item = {}
for user, item, _ in ratings(data):
    u_items = list(data[user].keys())
    sim_items = sim_whole[item][:k]
    intersect_items, index_u, _ = np.intersect1d(
        u_items, sim_items, assume_unique=True, return_indices=True)
    intersect_user_item[(user, item)] = (intersect_items, index_u)
    
intersect_user_item_test = {}
for user, item in zip(test_user_indices, test_item_indices):
    u_items = list(data[user].keys())
    sim_items = sim_whole[item][:k]
    intersect_items, index_u, _ = np.intersect1d(u_items, sim_items, assume_unique=True, return_indices=True)
    intersect_user_item_test[(user, item)] = (intersect_items, index_u)

In [12]:
def compute_rmse_neighbor_k(mode="train"):
    if mode == "train":
        user_indices = train_user_indices
        item_indices = train_item_indices
        ratings = train_ratings
        intersect = intersect_user_item
    elif mode == "test":
        user_indices = test_user_indices
        item_indices = test_item_indices
        ratings = test_ratings
        intersect = intersect_user_item_test
        
    pred_whole = []
    for u, i in zip(user_indices, item_indices):
        intersect_items, index_u = intersect[(u, i)]
        
        if tf.size(intersect_items).numpy() == 0:
            pred = global_mean + bu[u] + bi[i]
        else:
            u_ratings = np.array(list(data[u].values()))[index_u]
            base_neighbor = global_mean + bbu[u] + bbi[intersect_items]
            user_sqrt = tf.sqrt(tf.cast(tf.size(intersect_items), tf.float32))
            ru = tf.reduce_sum((u_ratings - base_neighbor) * tf.gather(tf.gather(w, i), intersect_items)) / user_sqrt
            nu = tf.reduce_sum(tf.gather(tf.gather(c, i), intersect_items)) / user_sqrt
            pred = global_mean + bu[u] + bi[i] + ru + nu
        pred_whole.append(pred)

    pred_whole = tf.convert_to_tensor(pred_whole)
    score = tf.sqrt(tf.reduce_mean(tf.pow(pred_whole - np.array(ratings), 2)))
    return score

In [13]:
np.random.seed(42)
n_factors = 100
k = 20  ###############
n_users = len(user2id)
n_items = len(item2id)
global_mean = np.mean(train_ratings)
base_user = bbu
base_item = bbi

n_epochs = 500
lr = 0.01
reg = 1e-4
display_step = 1

bu = tf.Variable(tf.zeros([n_users]))
bi = tf.Variable(tf.zeros([n_items]))
w = tf.Variable(tf.random_normal([n_items, n_items], 0.0, 0.01))
c = tf.Variable(tf.random_normal([n_items, n_items], 0.0, 0.01))

optimizer = tf.train.GradientDescentOptimizer(lr)
# optimizer = tf.train.AdamOptimizer(lr)

for epoch in range(n_epochs):
    t0 = time.time()
    with tf.GradientTape() as tape:
        variables = [bu, bi, w, c]
        loss = compute_rmse_neighbor_k()
        grads = tape.gradient(loss, variables)
        optimizer.apply_gradients(zip(grads, variables))
            
    print("Epoch: ", epoch + 1, "\ttime: ", time.time() - t0)
    
    if epoch % 1 == 0:
        t1 = time.time()
        print("Epoch: ", epoch + 1, "\ttraining RMSE: ", compute_rmse_neighbor_k("train").numpy(), 
              "\ttest RMSE: ", compute_rmse_neighbor_k("test").numpy())
        print("evaluate time: ", time.time() - t1)

Epoch:  1 	time:  80.52100014686584
Epoch:  1 	training RMSE:  1.1189075 	test RMSE:  1.1092225
evaluate time:  34.08999991416931
Epoch:  2 	time:  74.84100008010864
Epoch:  2 	training RMSE:  1.1189063 	test RMSE:  1.1092225
evaluate time:  32.74799990653992


KeyboardInterrupt: 

### Eager Execution - superSVD

In [9]:
import pickle
bbu, bbi = pickle.load(open("bbu_bbi.pkl", 'rb'))

In [10]:
for i in data:
    if len(data[i]) == 0:
        print(i)

In [11]:
def compute_rmse_superSVD(mode="train"):
    if mode == "train":
        user_indices = train_user_indices
        item_indices = train_item_indices
        ratings = train_ratings
        intersect = intersect_user_item
    elif mode == "test":
        user_indices = test_user_indices
        item_indices = test_item_indices
        ratings = test_ratings
        intersect = intersect_user_item_test
        
    nui = []
    for u in range(n_users):
        u_items = np.array(list(data[u].keys()))
        if tf.size(u_items).numpy() == 0:
            nui.append(tf.zeros(n_factors, dtype=tf.float32))
#            nui.append(np.zeros(100))
        else:
            nui.append(tf.reduce_sum(tf.gather(yj, u_items), axis=0) / tf.sqrt(tf.cast(tf.size(u_items), tf.float32)))
            
            
#    nui = tf.convert_to_tensor(nui)
    dot = tf.matmul(pu + nui, qi, transpose_b=True)
    indices = np.array([user_indices, item_indices]).T
    pred1 = global_mean + tf.gather(bu, user_indices) + tf.gather(bi, item_indices) + tf.gather_nd(dot, indices)
   
    
    pred2 = []
    for u, i in zip(user_indices, item_indices):
        intersect_items, index_u = intersect[(u, i)]
        
        if tf.size(intersect_items).numpy() == 0:
            pred2.append(0)
        else:
            u_ratings = np.array(list(data[u].values()))[index_u]
            base_neighbor = global_mean + bbu[u] + bbi[intersect_items]
            user_sqrt = tf.sqrt(tf.cast(tf.size(intersect_items), tf.float32))
            ru = tf.cast(tf.reduce_sum((u_ratings - base_neighbor) * tf.gather(tf.gather(w, i), intersect_items)), tf.float32) / user_sqrt
            nu = tf.cast(tf.reduce_sum(tf.gather(tf.gather(c, i), intersect_items)), tf.float32) / user_sqrt
            pred2.append(ru + nu)
            
    pred_whole = pred1 + pred2
    pred_whole = tf.convert_to_tensor(pred_whole)
    score = tf.sqrt(tf.reduce_mean(tf.pow(pred_whole - ratings, 2))) + \
            reg * (tf.nn.l2_loss(bu) + tf.nn.l2_loss(bi) + tf.nn.l2_loss(pu) + 
                   tf.nn.l2_loss(qi) + tf.nn.l2_loss(yj) + tf.nn.l2_loss(w) + tf.nn.l2_loss(c))
    return score

In [12]:
np.random.seed(42)
k = 20   ############################
n_users = len(user2id)
n_items = len(item2id)
n_factors = 100
global_mean = np.mean(train_ratings)
n_epochs = 200
lr = 0.001
reg = 0.01

bu = tf.Variable(tf.zeros([n_users,]))
bi = tf.Variable(tf.zeros([n_items,]))
pu = tf.Variable(tf.random_normal([n_users, n_factors], 0.0, 0.01))
qi = tf.Variable(tf.random_normal([n_items, n_factors], 0.0, 0.01))
yj = tf.Variable(tf.random_normal([n_items, n_factors], 0.0, 0.01))
w = tf.Variable(tf.random_normal([n_items, n_items], 0.0, 0.01))
c = tf.Variable(tf.random_normal([n_items, n_items], 0.0, 0.01))

# optimizer = tf.train.GradientDescentOptimizer(lr)
optimizer = tf.train.AdamOptimizer(lr)

for epoch in range(n_epochs):
    t0 = time.time()
    with tf.GradientTape() as tape:
        variables = [bu, bi, pu, qi, yj, w, c]
        loss = compute_rmse_superSVD()
        grads = tape.gradient(loss, variables)
        optimizer.apply_gradients(zip(grads, variables))
            
    print("Epoch: ", epoch + 1, "\ttime: ", time.time() - t0)
    
    if epoch % 1 == 0:
        t1 = time.time()
        print("Epoch: ", epoch + 1, "\ttraining RMSE: ", compute_rmse_superSVD("train").numpy(), 
              "\ttest RMSE: ", compute_rmse_superSVD("test").numpy())
        print("evaluate time: ", time.time() - t1)

Epoch:  1 	time:  2022.6800422668457
Epoch:  1 	training RMSE:  12.021564 	test RMSE:  12.021573
evaluate time:  35.197007179260254


SystemError: PyEval_EvalFrameEx returned a result with an error set

In [15]:
def compute_rmse_superSVD_batch(u, i, mode="train"):
    if mode == "train":
        user_indices = train_user_indices
        item_indices = train_item_indices
        ratings = train_ratings
        intersect = intersect_user_item
    elif mode == "test":
        user_indices = test_user_indices
        item_indices = test_item_indices
        ratings = test_ratings
        intersect = intersect_user_item_test

    u_items = np.array(list(data[u].keys()))
    nui = tf.reduce_sum(tf.gather(yj, u_items), axis=0) / tf.sqrt(tf.cast(tf.size(u_items), tf.float32))
            
            
#    nui = tf.convert_to_tensor(nui)
    dot = tf.reduce_sum(tf.multiply(tf.gather(pu, u) + nui, tf.gather(qi, i)))
#    indices = np.array([user_indices, item_indices]).T
    pred1 = global_mean + tf.gather(bu, u) + tf.gather(bi, i) + dot
   

    intersect_items, index_u = intersect[(u, i)]
    u_ratings = np.array(list(data[u].values()))[index_u]
    base_neighbor = global_mean + bbu[u] + bbi[intersect_items]
    user_sqrt = tf.sqrt(tf.cast(tf.size(intersect_items), tf.float32))
    ru = tf.cast(tf.reduce_sum((u_ratings - base_neighbor) * tf.gather(tf.gather(w, i), intersect_items)), tf.float32) / user_sqrt
    nu = tf.cast(tf.reduce_sum(tf.gather(tf.gather(c, i), intersect_items)), tf.float32) / user_sqrt
    pred2 = ru + nu
            
    pred_whole = pred1 + pred2
    pred_whole = tf.convert_to_tensor(pred_whole)
    score = tf.sqrt(tf.reduce_mean(tf.pow(pred_whole - ratings, 2))) + \
            reg * (tf.nn.l2_loss(bu) + tf.nn.l2_loss(bi) + tf.nn.l2_loss(pu) + 
                   tf.nn.l2_loss(qi) + tf.nn.l2_loss(yj) + tf.nn.l2_loss(w) + tf.nn.l2_loss(c))
    return score

In [16]:
np.random.seed(42)
k = 20   ############################
n_users = len(user2id)
n_items = len(item2id)
n_factors = 100
global_mean = np.mean(train_ratings)
n_epochs = 200
lr = 0.001
reg = 1.0

bu = tf.Variable(tf.zeros([n_users,]))
bi = tf.Variable(tf.zeros([n_items,]))
pu = tf.Variable(tf.random_normal([n_users, n_factors], 0.0, 0.01))
qi = tf.Variable(tf.random_normal([n_items, n_factors], 0.0, 0.01))
yj = tf.Variable(tf.random_normal([n_items, n_factors], 0.0, 0.01))
w = tf.Variable(tf.random_normal([n_items, n_items], 0.0, 0.01))
c = tf.Variable(tf.random_normal([n_items, n_items], 0.0, 0.01))

optimizer = tf.train.GradientDescentOptimizer(lr)
# optimizer = tf.train.AdamOptimizer(lr)

for epoch in range(n_epochs):
    t0 = time.time()
    for u, i in zip(train_user_indices, train_item_indices):
        with tf.GradientTape() as tape:
            variables = [bu, bi, pu, qi, yj, w, c]
            loss = compute_rmse_superSVD_batch(u, i)
            grads = tape.gradient(loss, variables)
            optimizer.apply_gradients(zip(grads, variables))
            
    print("Epoch: ", epoch + 1, "\ttime: ", time.time() - t0)
    
    if epoch % 1 == 0:
        t1 = time.time()
        print("Epoch: ", epoch + 1, "\ttraining RMSE: ", compute_rmse_superSVD("train").numpy(), 
              "\ttest RMSE: ", compute_rmse_superSVD("test").numpy())
        print("evaluate time: ", time.time() - t1)

Epoch:  1 	time:  570.9743838310242
Epoch:  1 	training RMSE:  nan 	test RMSE:  nan
evaluate time:  4.376272678375244


KeyboardInterrupt: 